In [2]:
# RUN THIS ONLY ONCE TO CREATE THE TABLES FOR THE DEMO (IMPRESSIONSTOORDERS)!!!!!
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

In [3]:
import json

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)


In [4]:
# generate fake impressions-orders events for demo
import pyodbc 
from dotenv import load_dotenv
import os
load_dotenv()

SQL_SERVER = os.getenv("SQL_SERVER") 
SQL_USER = os.getenv("SQL_USER")
SQL_PWD = os.getenv("SQL_PWD")
SQL_DBNAME = os.getenv("SQL_DBNAME")
SQL_DRIVER = "SQL SERVER"
connectionString = f'DRIVER={SQL_DRIVER};SERVER={SQL_SERVER};DATABASE={SQL_DBNAME};UID={SQL_USER};PWD={SQL_PWD}'
conn = pyodbc.connect(connectionString)

import pandas as pd
sql_query = """
  SELECT  [SalesOrderID],[SalesOrderDetailID],[ProductID]
  FROM [SalesLT].[SalesOrderDetail]
"""

df = pd.read_sql_query(sql_query, conn)

C:\Users\dschlesinger\AppData\Local\Temp\ipykernel_34676\2410531013.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


In [ ]:
from utils import generateClickEvent, generateImpressionEvent

impressionsToOrdersDF = pd.DataFrame(columns=['impressionId', 'SalesOrderID', 'SalesOrderDetailID'])
for index, row in df.iterrows():
    SalesOrderID = row['SalesOrderID']
    SalesOrderDetailID = row['SalesOrderDetailID']
    productId = row['ProductID']
    
    #generate click and impression events
    impressionEvent = generateImpressionEvent(False, False, productId)
    sendToEventsHub(impressionEvent, producer_impressions)
    clickEvent = generateClickEvent(impressionEvent)
    sendToEventsHub(clickEvent, producer_clicks)

    #generate impressionToOrder event
    impressionToOrder = {}
    impressionToOrder["impressionId"] = impressionEvent["impressionId"]
    impressionToOrder["salesOrderId"] = SalesOrderID
    impressionToOrder["salesOrderDetailId"] = SalesOrderDetailID
    impressionsToOrdersDF.loc[len(impressionsToOrdersDF)] = impressionToOrder
impressionsToOrdersDF.to_csv("./data/impressionsToOrders.csv", index=False)
producer_impressions.close()
producer_clicks.close()

In [ ]:
import pandas as pd
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
from azure.kusto.data.data_format import DataFormat
from azure.kusto.ingest import (
    BlobDescriptor,
    FileDescriptor,
    IngestionProperties,
    IngestionStatus,
    KustoStreamingIngestClient,
    ManagedStreamingIngestClient,
    QueuedIngestClient,
    StreamDescriptor,
)
load_dotenv()
AAD_TENANT_ID = os.getenv("AAD_TENANT_ID")
KUSTO_CLUSTER = os.getenv("KUSTO_CLUSTER")
KUSTO_INGEST_URI = os.getenv("KUSTO_INGEST_URI")
KUSTO_DATABASE = os.getenv("KUSTO_DATABASE")
KUSTO_MANAGED_IDENTITY_APP_ID = os.getenv("KUSTO_MANAGED_IDENTITY_APP_ID")
KUSTO_MANAGED_IDENTITY_SECRET = os.getenv("KUSTO_MANAGED_IDENTITY_SECRET")

# Connect to adx using AAD app registration for ingestion - use ingest URI
kcsb = KustoConnectionStringBuilder.with_aad_application_key_authentication(KUSTO_INGEST_URI, KUSTO_MANAGED_IDENTITY_APP_ID, KUSTO_MANAGED_IDENTITY_SECRET,  AAD_TENANT_ID)
client = KustoClient(kcsb)
kusto_db = KUSTO_DATABASE

DESTINATION_TABLE = "impressionsToOrders"
DESTINATION_TABLE_COLUMN_MAPPING = "impressionsToOrders_mapping"

client = QueuedIngestClient(kcsb)

# there are a lot of useful properties, make sure to go over docs and check them out
ingestion_props = IngestionProperties(
    database=KUSTO_DATABASE,
    table=DESTINATION_TABLE,
    data_format=DataFormat.CSV,
    # in case status update for success are also required (remember to import ReportLevel from azure.kusto.ingest)
    # report_level=ReportLevel.FailuresAndSuccesses,
    # in case a mapping is required (remember to import IngestionMappingKind from azure.kusto.data.data_format)
    # ingestion_mapping_reference="{json_mapping_that_already_exists_on_table}",
    # ingestion_mapping_kind= IngestionMappingKind.JSON,
)

df = pd.read_csv("./data/impressionsToOrders.csv")

client.ingest_from_dataframe(df, ingestion_properties=ingestion_props)